In [ ]:
"""
다중 텍스트 프롬프트 기반 객체 탐지 예제
===============================================
이 노트북은 각 클래스별로 여러 개의 텍스트 프롬프트를 사용한 객체 탐지 예제입니다.
다양한 프롬프트를 통해 탐지 성능을 높이려 했으나, 실제로는 프롬프트 품질에 따른 
성능 편차가 크다는 한계가 발견되어 임베딩 기반 솔루션으로 전환되었습니다.

"""

import os
import json
from pathlib import Path

import torch
from torchvision.ops import nms
from transformers import Owlv2Processor, Owlv2ForObjectDetection
from PIL import Image, ImageDraw, ImageFont

# ==================== 디렉토리 설정 ====================
IMAGE_DIR   = Path("encumbrance_labels/encumbrance")           # 원본 이미지 디렉토리
OUTPUT_DIR  = Path("encumbrance_labels/prediction_with_multiple_text")  # 결과 저장 디렉토리

# ==================== 하이퍼파라미터 ====================
TARGET_SIZE    = 1008    # OWLv2 최적 입력 크기 (14×72 패치)
TEXT_THRESHOLD = 0.3     # 텍스트-이미지 매칭 신뢰도 임계값
IOU_THRESHOLD  = 0.5     # NMS IoU 임계값

# ==================== 디바이스 설정 ====================
# Apple Silicon (MPS) > NVIDIA GPU (CUDA) > CPU 순서로 선택
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")  # Apple Silicon GPU
else:
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # NVIDIA GPU 또는 CPU

print(f"Using device: {DEVICE}")

# ==================== 프롬프트 전략 ====================
# 클래스별 다양한 영어 프롬프트 5개씩 정의
# 각 클래스마다 서로 다른 표현을 사용하여 탐지 성능 향상 시도
QUERY_PROMPTS = {
    "Tomb": [  # 무덤/고분 클래스
        "Korean burial mound",
        "ancient tomb site",
        "grave mound",
        "traditional burial mound",
        "burial site covered with grass"
    ],
    "Tree": [
        "isolated tree canopy",
        "deciduous tree",
        "single tall tree",
        "green leafed tree",
        "mature oak tree"
    ],
    "Greenhouse": [
        "plastic greenhouse",
        "glass greenhouse structure",
        "hoop house greenhouse",
        "abandoned greenhouse frame",
        "greenhouse with plastic cover"
    ],
    "Building": [
        "single story building",
        "flat roof house",
        "residential building structure",
        "industrial shed building",
        "roofed concrete building"
    ],
    "Field": [
        "agricultural field",
        "plowed farmland",
        "cultivated crop field",
        "rice paddy field",
        "open grass field"
    ],
    "Container": [
        "shipping container",
        "cargo container",
        "metal storage container",
        "freight container box",
        "stacked container unit"
    ]
}

# 클래스별 시각화 색상 (RGB)
COLOR_MAP = {
    "Tomb":       (  0,   0, 255),
    "Tree":       (  0, 255,   0),
    "Greenhouse": (255,   0,   0),
    "Building":   (  0, 255, 255),
    "Field":      (255,   0, 255),
    "Container":  (255, 255,   0),
}
# ────────────────────────────────────────────────────────────

# ==================== 모델 초기화 ====================
# 1) OWLv2 모델 & 프로세서 로드 
processor = Owlv2Processor.from_pretrained(
    "google/owlv2-large-patch14-ensemble",
    image_size=TARGET_SIZE  # 1008x1008 입력 크기
)
model = Owlv2ForObjectDetection.from_pretrained(
    "google/owlv2-large-patch14-ensemble"
).to(DEVICE).eval()  # 평가 모드로 설정

# 2) 출력 폴더 준비
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
font = ImageFont.load_default()  # 텍스트 표시용 폰트

# ==================== 메인 처리 루프 ====================
# 3) 이미지별 처리 - 각 이미지에 대해 다중 프롬프트 탐지 수행
for img_path in IMAGE_DIR.glob("*"):
    # 지원하는 이미지 형식만 처리
    if img_path.suffix.lower() not in {".jpg", ".jpeg", ".png"}:
        continue

    print(f"\n▶ Processing {img_path.name} …")
    
    # ==================== 이미지 전처리 ====================
    # 원본 이미지 로드 및 RGB 변환
    orig = Image.open(img_path).convert("RGB")
    # OWLv2 모델 입력 크기로 리사이징
    img  = orig.resize((TARGET_SIZE, TARGET_SIZE), Image.BILINEAR)
    w, h = img.size  # w == h == 1008

    # ==================== 다중 프롬프트 탐지 결과 저장용 리스트 ====================
    all_boxes, all_scores, all_labels = [], [], []

    # ==================== 4단계: 클래스별 다중 텍스트 프롬프트 탐지 ====================
    # 각 클래스마다 5개의 서로 다른 프롬프트로 탐지 수행
    for cls, prompts in QUERY_PROMPTS.items():
        print(f"  클래스 '{cls}' 탐지 중... ({len(prompts)}개 프롬프트)")
        
        # ==================== OWLv2 모델 입력 준비 ====================
        # 다중 프롬프트와 단일 이미지를 프로세서에 전달
        inputs = processor(
            text=prompts,      # 5개의 서로 다른 프롬프트
            images=img,        # 동일한 이미지
            return_tensors="pt"
        )
        
        # GPU로 데이터 이동 (가능한 경우)
        for k, v in inputs.items():
            if isinstance(v, torch.Tensor):
                inputs[k] = v.to(DEVICE)

        # ==================== 모델 추론 및 후처리 ====================
        # GPU 메모리 사용량 최적화를 위한 no_grad 컨텍스트
        with torch.no_grad():
            outputs = model(**inputs)  # OWLv2 모델 추론
            
        # 모델 출력을 바운딩 박스, 점수, 라벨로 변환
        results = processor.post_process_object_detection(
            outputs=outputs,
            target_sizes=torch.tensor([[h, w]], device=DEVICE),  # 원본 이미지 크기
            threshold=TEXT_THRESHOLD  # 신뢰도 임계값 적용
        )

        # ==================== 결과 수집 ====================
        # 각 프롬프트별 탐지 결과를 전체 리스트에 추가
        for res in results:
            for box, score in zip(res["boxes"], res["scores"]):
                all_boxes.append(box)      # 바운딩 박스 좌표
                all_scores.append(score)   # 신뢰도 점수
                all_labels.append(cls)     # 클래스명

    # ==================== 5단계: 전역 Non-Maximum Suppression (NMS) ====================
    # 모든 클래스의 탐지 결과를 통합하여 중복 제거
    if not all_boxes:
        print(f"  [WARN] No detections for {img_path.name}")
        continue

    # 텐서로 변환
    boxes_t  = torch.stack(all_boxes)   # 모든 바운딩 박스를 하나의 텐서로
    scores_t = torch.stack(all_scores)  # 모든 점수를 하나의 텐서로
    
    # NMS 적용: 겹치는 박스들 중에서 가장 높은 점수의 박스만 유지
    keep     = nms(boxes_t, scores_t, IOU_THRESHOLD)

    # ==================== 6단계: 시각화 및 JSON 작성 ====================
    draw = ImageDraw.Draw(img)  # 이미지에 그리기 객체 생성
    records = []  # JSON 저장용 결과 리스트

    # NMS로 필터링된 최종 탐지 결과에 대해 시각화
    for i in keep:
        # ==================== 박스 좌표 처리 ====================
        x1, y1, x2, y2 = boxes_t[i].tolist()
        # 이미지 경계를 벗어나는 박스 좌표를 이미지 내부로 제한
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w, x2), min(h, y2)
        
        # 클래스 정보 및 색상 설정
        label = all_labels[i]  # 클래스명
        score = float(scores_t[i])  # 신뢰도 점수
        color = COLOR_MAP.get(label, (255,255,255))  # 클래스별 색상

        # ==================== 바운딩 박스 그리기 ====================
        draw.rectangle([x1, y1, x2, y2], outline=color, width=3)
        
        # ==================== 라벨 텍스트 표시 ====================
        txt = f"{label}: {score:.2f}"  # 클래스명과 신뢰도 점수
        # 텍스트 배경 박스 계산
        tx0, ty0, tx1, ty1 = draw.textbbox((x1, y1), txt, font=font)
        # 텍스트 배경 그리기
        draw.rectangle([tx0, ty0, tx1, ty1], fill=color)
        # 흰색 텍스트 그리기
        draw.text((x1, y1), txt, fill=(255,255,255), font=font)

        # ==================== JSON 기록용 데이터 추가 ====================
        records.append({
            "label": label,
            "score": score,
            "box": [round(x1,2), round(y1,2), round(x2,2), round(y2,2)]
        })

    # ==================== 7단계: 결과 파일 저장 ====================
    out_img  = OUTPUT_DIR / img_path.name  # 시각화된 이미지 저장 경로
    out_json = OUTPUT_DIR / f"{img_path.stem}.json"  # JSON 결과 저장 경로
    
    # 시각화된 이미지 저장
    img.save(out_img)
    # JSON 결과 저장 (UTF-8 인코딩, 한글 지원)
    with open(out_json, "w", encoding="utf-8") as fp:
        json.dump(records, fp, ensure_ascii=False, indent=2)

    print(f"  [OK] Saved → {out_img.name}, {out_json.name}")


/home/dromii4/dromii/owlv2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



▶ Processing DJI_0951.JPG …
  [OK] Saved → DJI_0951.JPG, DJI_0951.json

▶ Processing DJI_0691.JPG …
  [OK] Saved → DJI_0691.JPG, DJI_0691.json

▶ Processing DJI_0033.JPG …
  [OK] Saved → DJI_0033.JPG, DJI_0033.json

▶ Processing DJI_0559.jpg …
  [OK] Saved → DJI_0559.jpg, DJI_0559.json

▶ Processing DJI_0934.jpg …
  [OK] Saved → DJI_0934.jpg, DJI_0934.json

▶ Processing DJI_0923.jpg …
  [OK] Saved → DJI_0923.jpg, DJI_0923.json

▶ Processing DJI_0008.JPG …
  [OK] Saved → DJI_0008.JPG, DJI_0008.json

▶ Processing DJI_0933.jpg …
  [OK] Saved → DJI_0933.jpg, DJI_0933.json

▶ Processing DJI_0866.JPG …
  [OK] Saved → DJI_0866.JPG, DJI_0866.json

▶ Processing DJI_0036.JPG …
  [OK] Saved → DJI_0036.JPG, DJI_0036.json

▶ Processing DJI_0928.jpg …
  [OK] Saved → DJI_0928.jpg, DJI_0928.json
